## In-depth Introduction
First let's import the module and create an environment.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import loadtxt
import tensorflow as tf

import keras
import pandas as pd
import numpy as np
import riiideducation

# You can only call make_env() once, so don't lose it!
#env = riiideducation.make_env()

In [ ]:
questions = pd.read_csv("../input/userquestions-dataframes-csv/questions_dataframe.csv")
users = pd.read_csv("../input/userquestions-dataframes-csv/users_dataframe.csv")

In [ ]:
users

In [ ]:
model = keras.models.load_model("../input/model2/wholedatamodel_5layers_256r_128r_64r_32r_1s_lr.00001.h5")

In [ ]:
#p = np.array([1.20100000e+04,8.30090791e-01,7.09174841e-01,1,0])

In [ ]:
#p = p.reshape(-1,1)

In [ ]:
#p.shape

In [ ]:
#model.predict([ 5.69200000e+03,  0.00000000e+00,  0.00000000e+00,
#         7.45494879e-01,  6.95652174e-01])

In [ ]:
model.summary()

In [ ]:
feature_list = ['content_id','prior_question_elapsed_time','prior_question_had_explanation', 'question_correct_ratio','student_correct_ratio']

In [ ]:
def prepare_test_features(test_df,feature_list):
    ['content_id','prior_question_elapsed_time','prior_question_had_explanation', 'question_correct_ratio','student_correct_ratio']
    
        
    test_df = pd.merge(test_df,questions[['content_id', 'question_correct_ratio']],how='left',on='content_id')
    test_df = pd.merge(test_df,users[['user_id','student_correct_ratio']],how='left',on='user_id')
    test_df['prior_question_elapsed_time'].fillna(-1,inplace = True)
    test_df['prior_question_had_explanation'] *= 1 # convert from boolean to numbers
    test_df['prior_question_had_explanation'].fillna(-1,inplace = True)
    test_df.fillna(-1,inplace = True)
    return test_df[feature_list].to_numpy().astype(float)

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_questions = test_df['content_id'].to_numpy()
    #print("test_questions: ", test_questions)
    test_users = test_df['user_id'].to_numpy()
    #print("test_users: ", test_users)
    test_set = prepare_test_features(test_df,feature_list)
    #print("test_set: ", test_set)
    answered_correctly = model.predict_classes(test_set)
    #print("answered_correctly: ", answered_correctly)
    #answered_correctly = clf.predict(test_set)
    test_df['answered_correctly'] = answered_correctly
    env.predict(test_df.loc[test_df['content_type_id']==0,['row_id','answered_correctly']])